In [10]:
import nltk
nltk.download()
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import csv
import re
import string
from collections import Counter
import pprint

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


pre processing - start

In [11]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', '__']
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs
    # r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    # r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

Pre-Processing functions

In [12]:
def tokenize(s):
    s = re.sub(r'[^\x00-\x7f]*',r'',s)
    return tokens_re.findall(s)

def cleanAndNormalizeText(data):
    tokens = tokenize(data)
    tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    # tokenize
    # tokenize = word_tokenize(data)
    # remove stop words
    filterText = [w for w in tokens if w not in stop]
    filterText = [w for w in filterText if not len(w) <= 1]
    # stem
    ps = PorterStemmer()
    for i in range(len(filterText) - 1):
        if len(filterText[i]) > 1:
            try:
                filterText[i] = ps.stem(filterText[i])
            except Exception as e:
                filterText[i] = filterText[i]

    return filterText

Downloading the data ,clean and save terms per gender

In [13]:
terms_male = []
terms_female = []
terms_brand = []
genderClassData = []
with open('C:\\Users\\abalata\\Downloads\\gender-classifier-DFE-791531.csv\\gender-classifier-DFE-791531.csv','r', encoding="utf8" ,errors="ignore") as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        if count != 0:
            text = row[10]+' '+row[13]+' '+row[18]+' '+row[19]
            cleantext = cleanAndNormalizeText(text)
            genderClassData.append([row[0], row[5], ' '.join(cleantext)])
            for token in cleantext:
                if token not in stop:
                    if row[5] == 'male':
                        terms_male.append(token)
                    else:
                        if row[5] == 'female':
                            terms_female.append(token)
                        else:
                            if row[5] == 'brand':
                                terms_brand.append(token)
        count = count+1


Table = pd.DataFrame(genderClassData, columns=['ID', 'gender', 'text'])

In [15]:
classDistribution = []
for c in ['male', 'female', 'brand']:
    classDistribution.append([c, Table["gender"].value_counts()[c]])
pd.DataFrame(classDistribution, columns=['Gender', '# of Samples'])


,Gender,# of Samples
0,male,6194
1,female,6700
2,brand,5942


Term freqency

In [17]:
count_male = Counter()
count_male.update(terms_male)
pp = pprint.PrettyPrinter()
print("Male most common:",sum(count_male.values()))
pp.pprint(count_male.most_common(20))


count_female = Counter()
count_female.update(terms_female)
print("Female most common:",sum(count_female.values()))
pp.pprint(count_female.most_common(20))

count_brand = Counter()
count_brand.update(terms_brand)
print("Brand most common:",sum(count_brand.values()))
pp.pprint(count_brand.most_common(20))

Male most common: 117730
[('0084b4', 3090),
 ('c0deed', 2769),
 ('ffffff', 1152),
 ('love', 668),
 ('like', 575),
 ('get', 558),
 ('one', 423),
 ('go', 409),
 ('time', 400),
 ('make', 398),
 ('life', 367),
 ('fan', 335),
 ('follow', 331),
 ('day', 313),
 ('know', 290),
 ('new', 286),
 ('9999', 284),
 ('good', 276),
 ('music', 275),
 ('peopl', 273)]
Female most common: 113958
[('0084b4', 2652),
 ('c0deed', 2483),
 ('ffffff', 1512),
 ('love', 973),
 ('like', 711),
 ('get', 555),
 ('one', 508),
 ('make', 503),
 ('go', 488),
 ('life', 457),
 ('day', 455),
 ('time', 434),
 ('follow', 405),
 ('girl', 354),
 ('want', 336),
 ('know', 318),
 ('peopl', 316),
 ('live', 310),
 ('thing', 296),
 ('new', 292)]
Brand most common: 113865
[('0084b4', 3441),
 ('c0deed', 3157),
 ('weather', 2327),
 ('get', 1513),
 ('updat', 1379),
 ('channel', 1218),
 ('15', 1195),
 ('ffffff', 969),
 ('40', 740),
 ('news', 604),
 ('39', 425),
 ('us', 421),
 ('follow', 419),
 ('new', 387),
 ('love', 313),
 ('make', 298),
 

In [18]:

# split to train and test

from sklearn.model_selection import train_test_split
x = Table.drop(['gender'], axis=1)
y = Table[["gender"]]
print(x)
print(y)

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=0)
xTrain = xTrain.apply(lambda x: " ".join(x), axis=1)
xTest = xTest.apply(lambda x: " ".join(x), axis=1)
xTrain = xTrain.tolist()
xTest = xTest.tolist()
yTrain = yTrain['gender'].tolist()
yTest = yTest['gender'].tolist()

              ID                                               text
0      815719226  sing rhythm 08c2c2 ffffff robbi respond critic...
1      815719227  author novel fill famili drama romanc 0084b4 c...
2      815719228  loui whine squeal abb8c2 c0deed absolut ador l...
3      815719229  mobil guy 49er shazam googl kleiner perkin yah...
4      815719230  ricki wilson best frontman kaiser chief best b...
5      815719231  know f5abb5 ive seen peopl train lamp chair tv...
6      815719232  global marketplac imag video music share photo...
7      815719233  secret get ahead get start 0000ff c0deed gala ...
8      815719234  pll fan crazi mcd ramen bae 9266cc @_aphmau_ p...
9      815719235  renaiss art historian univers nottingham fuell...
10     815719236  clean food tast great provid energi nutrient g...
11     815719237  highli extraordinari auction 0084b4 c0deed mtg...
12     815719238  senior 16 xi xii mmxiv 0084b4 ffffff put ass l...
13     815719239  come join fastest blog network

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# The classifying process
def classify(parameters, feature, ml):
            pipeline = Pipeline(steps=[('vect', feature), ('clf', ml)])
            gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
            gs_clf = gs_clf.fit(xTrain, yTrain)
            prediction = gs_clf.predict(xTest)
            accuracy = metrics.accuracy_score(yTest, prediction)
            return [accuracy, gs_clf.best_params_]

In [ ]:

# keras model

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

#create the model
embedding_vecor_length = 32
max_review_length = 1000
model = Sequential()
model.add(Embedding(10, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=128)



results = []
scores = model.evaluate(xTest, yTest, verbose=0)
results.append(['****', '***', '***', (scores[1]*100)])

In [20]:

# other models and show the accuracy

parameters = {'vect__max_df': (0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0), 'clf__alpha': (0.001, 0.01, 0.1, 1.0)}

# Using classify with Bag of Words and SVM algorithm
ans = classify(parameters, CountVectorizer(), SGDClassifier())
results.append(['Bag of Words', 'SVM', ans[1], ans[0]])

# Using classify with Bag of Words and NB algorithm
ans = classify(parameters, CountVectorizer(), MultinomialNB())
results.append(['Bag of Words', 'Naïve Bayes',  ans[1], ans[0]])

# Using classify with tf-idf and SVM algorithm
ans = classify(parameters, TfidfVectorizer(sublinear_tf=True, stop_words='english'), SGDClassifier())
results.append(['TF-IDF', 'SVM', ans[1], ans[0]])

# Using classify with tf-idf and NB algorithm
ans = classify(parameters, TfidfVectorizer(sublinear_tf=True, stop_words='english'), MultinomialNB())
results.append(['TF-IDF', 'Naïve Bayes', ans[1], ans[0]])

pd.DataFrame(results, columns=['feature extract', 'machine learning algorithm', 'parameters', 'accuracy'])

C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


...

C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


,feature extract,machine learning algorithm,parameters,accuracy
0,Bag of Words,SVM,"{'clf__alpha': 0.01, 'vect__max_df': 0.7}",0.592020
1,Bag of Words,Naïve Bayes,"{'clf__alpha': 1.0, 'vect__max_df': 0.4}",0.608978
2,TF-IDF,SVM,"{'clf__alpha': 0.001, 'vect__max_df': 0.8}",0.592851
3,TF-IDF,Naïve Bayes,"{'clf__alpha': 1.0, 'vect__max_df': 0.4}",0.596509
